<a href="https://colab.research.google.com/github/TarunjitYumnam/Large-Language-Model/blob/main/Peft_Lora.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import transformers

In [ ]:
from transformers import GPT2Tokenizer
from transformers import GPT2LMHeadModel

In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2-large')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/666 [00:00<?, ?B/s]

In [ ]:
model = GPT2LMHeadModel.from_pretrained('gpt2-large',pad_token_id = tokenizer.eos_token_id)

model.safetensors:   0%|          | 0.00/3.25G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
from peft import get_peft_model,LoraConfig

In [ ]:
lora_config = LoraConfig(r=16,
                         lora_alpha=32,
                         lora_dropout=0.1,
                         task_type = 'CAUSAL_LM')

In [ ]:
peft_model = get_peft_model(model,lora_config)

/usr/local/lib/python3.10/dist-packages/peft/tuners/lora/layer.py:1264: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


In [ ]:
peft_model.train()

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): GPT2LMHeadModel(
      (transformer): GPT2Model(
        (wte): Embedding(50257, 1280)
        (wpe): Embedding(1024, 1280)
        (drop): Dropout(p=0.1, inplace=False)
        (h): ModuleList(
          (0-35): 36 x GPT2Block(
            (ln_1): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
            (attn): GPT2SdpaAttention(
              (c_attn): lora.Linear(
                (base_layer): Conv1D(nf=3840, nx=1280)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=1280, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=3840, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()


In [ ]:
text = """ This Machine Learning with Python course dives into the basics of machine learning using an approachable and well-known programming language.
You'll learn about Supervised vs. Unsupervised Learning, look into how Statistical Modeling relates to Machine Learning, and do a comparison of each.
Look at real-life examples of Machine learning and how it affects society in ways you may not have guessed!Please note that this course has some hands-on
labs which requires that you have working knowledge of Python programming language. If you don't feel you have sufficient skill in Python programming,
I recommend you to take the Python """
encoded_text = tokenizer(text,return_tensors='pt')
encoded_text

{'input_ids': tensor([[  770, 10850, 18252,   351, 11361,  1781, 46638,   656,   262, 19165,
           286,  4572,  4673,  1262,   281,  3164,   540,   290,   880,    12,
          4002,  8300,  3303,    13,   198,  1639,  1183,  2193,   546,  3115,
         16149,  3691,    13,   791, 16668, 16149, 18252,    11,   804,   656,
           703, 34931,  9104,   278, 18436,   284, 10850, 18252,    11,   290,
           466,   257,  7208,   286,  1123,    13,   198,  8567,   379,  1103,
            12,  6042,  6096,   286, 10850,  4673,   290,   703,   340, 10975,
          3592,   287,  2842,   345,   743,   407,   423, 25183,     0,  5492,
          3465,   326,   428,  1781,   468,   617,  2832,    12,   261,   198,
            75,  8937,   543,  4433,   326,   345,   423,  1762,  3725,   286,
         11361,  8300,  3303,    13,  1002,   345,   836,   470,  1254,   345,
           423,  6751,  5032,   287, 11361,  8300,    11,   198,    40,  4313,
           345,   284,  1011,   262, 1

In [ ]:
import torch
optimizer = torch.optim.AdamW(peft_model.parameters())
for epoch in range(10):
  output = peft_model(input_ids=encoded_text['input_ids'],attention_mask=encoded_text['attention_mask'],labels=encoded_text['input_ids'])
  loss = output.loss
  loss.backward()
  optimizer.step()
  optimizer.zero_grad()
  print(f"Epoch: {epoch+1} ========== Loss: {loss.item()}")

Epoch: 1 ========== Loss: 2.9960386753082275
Epoch: 2 ========== Loss: 2.8554317951202393
Epoch: 3 ========== Loss: 2.569777250289917
Epoch: 4 ========== Loss: 2.226706027984619
Epoch: 5 ========== Loss: 1.9767954349517822
Epoch: 6 ========== Loss: 1.6173170804977417
Epoch: 7 ========== Loss: 1.2724850177764893
Epoch: 8 ========== Loss: 0.9840465784072876
Epoch: 9 ========== Loss: 0.6810572743415833
Epoch: 10 ========== Loss: 0.46427544951438904


In [ ]:
peft_model.eval()
input = "help me understanding machine learning"
encoded_text = tokenizer.encode(input,return_tensors = 'pt')
result = peft_model.generate(encoded_text,max_length=200,num_beams = 5, no_repeat_ngram_size=2,early_stopping=True)
result

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


tensor([[16794,   502,  4547,  4572,  4673,   290,   703,   340, 18436,   284,
           262,  1103,   995,   526,   198,   198,   464,  1781,  8698,   262,
         19165,   286, 10850, 18252,  1262,   281,  3164,   540,   290,   880,
            12,  4002,  3115, 16149,  3691,    13,   791, 16668, 16149, 18252,
          9104,    13,   921,  1183,   635,  2193,   546, 34931,  9104,   278,
            11,   804,   379,   703, 10850,  4673, 10975,  3592,   287,  2842,
           345,   743,   407,   423, 25183,    11,   290,  1011,   257,   804,
           656,   703,   262,  9104, 23008,   284,  1103,    12,  6042,  6096,
           287,  7208,   284,   703,   345,  1244,  1607,   340,   284,  1620,
            13,    40,   804,  2651,   284,   534,  7538,   319,   262,  1781,
           290,   314,  2911,   345,  1183,  1011,   262,   640,   284,  1577,
           502,   534,  6066,   319,   703,   314,  1053,  1760,   616,  1693,
           880,     0,   198,    40,  1549,   635,  

In [ ]:
result = tokenizer.decode(result[0],skip_special_tokens=True)
result

'help me understanding machine learning and how it relates to the real world."\n\nThe course covers the basics of Machine Learning using an approachable and well-known Supervised vs. Unsupervised Learning Model. You\'ll also learn about Statistical Modeling, look at how Machine learning affects society in ways you may not have guessed, and take a look into how the Model compares to real-life examples in comparison to how you might expect it to perform.I look forward to your feedback on the course and I hope you\'ll take the time to give me your thoughts on how I\'ve done my job well!\nI\'d also like to take this opportunity to thank you for your interest in my course. If you have any questions or comments, please feel free to contact me via email at [email protected]'